# Upload nuclei/soma table to CAVE
## Prepare CAVE environment

In [1]:
import numpy as np
import sys
import os
import pandas as pd
from datetime import datetime
from caveclient import CAVEclient

sys.path.append(os.path.abspath("../segmentation"))# to import rootID_lookup and authentication_utils like below
import rootID_lookup as IDlook
import authentication_utils as auth

In [2]:
# auth = client.auth
# auth.get_new_token()
# new_token = '' # This is the text you see after you visit the website.
# auth.save_token(token=new_token)

In [3]:
# check datastack_name here https://globalv1.daf-apis.com/info/
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.info.get_datastack_info()

{'description': 'This is the first version of the fanc dataset hosted in CAVE.',
 'aligned_volume': {'description': 'This is the 4th alignment of the female adult nerve cord dataset.',
  'name': 'fanc_v4',
  'id': 7,
  'image_source': 'precomputed://gs://zetta_lee_fly_vnc_001_precomputed/fanc_v4_em'},
 'synapse_table': None,
 'segmentation_source': 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod',
 'viewer_site': None,
 'local_server': 'https://cave.fanc-fly.com',
 'analysis_database': None,
 'soma_table': None}

In [4]:
client.annotation.get_tables()

https://cave.fanc-fly.com/annotation/api/v2/aligned_volume/fanc_v4/table


['test_soma',
 'synapses_may2021',
 'nuclei_aug2021',
 'soma_aug2021',
 't1mn_not_valid',
 'nuclei_aug2021ver2']

## format tables

In [9]:
df = pd.read_csv('../Output/soma_info_Aug2021ver5.csv', header=0)

In [10]:
# https://globalv1.daf-apis.com/schema/views/type/nucleus_detection/view
nuclei_df = df.reindex(columns=['nucID', 'nuc_xyz', 'voxel_size', 'bbx_min', 'bbx_max'])
nuclei_df = nuclei_df.rename(columns={'nucID': 'id', 'nuc_xyz': 'pt_position', 'voxel_size': 'volume', 'bbx_min': 'bb_start_position', 'bbx_max': 'bb_end_position'})
nuclei_df['type'] = 'nucleus_detection'
nuclei_df['volume'] = nuclei_df['volume'] * 0.0688*0.0688*0.045 # um^3

soma_df = df.reindex(columns=['nucID', 'soma_xyz'])
soma_df = soma_df.rename(columns={'nucID': 'id', 'soma_xyz': 'pt_position'})
soma_df['type'] = 'nucleus_detection'

In [11]:
nuclei_df_pt = nuclei_df['pt_position'].str.strip('()').str.split(',',expand=True)
nuclei_df['pt_position'] = nuclei_df_pt.astype(int).values.tolist()

nuclei_df_bb_start = nuclei_df['bb_start_position'].str.strip('()').str.split(',',expand=True)
nuclei_df['bb_start_position'] = nuclei_df_bb_start.astype(int).values.tolist()

nuclei_df_bb_end = nuclei_df['bb_end_position'].str.strip('()').str.split(',',expand=True)
nuclei_df['bb_end_position'] = nuclei_df_bb_end.astype(int).values.tolist()

In [12]:
soma_df_pt = soma_df['pt_position'].str.strip('()').str.split(',',expand=True)
soma_df['pt_position'] = soma_df_pt.astype(int).values.tolist()

In [13]:
# check tables
nuclei_df

,id,pt_position,volume,bb_start_position,bb_end_position,type
0,73115668492321345,"[64384, 155376, 2719]",2.905385,"[64144, 155296, 2711]","[64544, 155760, 2756]",nucleus_detection
1,72341681025843555,"[16960, 157296, 2652]",7.035123,"[16832, 157232, 2639]","[17216, 157648, 2692]",nucleus_detection
2,72694142819371754,"[40352, 195888, 1211]",3.313929,"[40224, 195760, 1197]","[40672, 196240, 1238]",nucleus_detection
3,72904562259788369,"[52752, 154288, 2630]",4.187674,"[52368, 153920, 2624]","[52848, 154320, 2671]",nucleus_detection
4,72481250081768943,"[27376, 86896, 2005]",2.807190,"[27008, 86544, 1982]","[27408, 87104, 2023]",nucleus_detection
...,...,...,...,...,...,...
17071,73114912242532398,"[60624, 109584, 1417]",273.849408,"[59856, 108960, 1351]","[62560, 112032, 1522]",nucleus_detection
17072,72903943314735186,"[48496, 116896, 969]",371.881246,"[47664, 116336, 828]","[50896, 118912, 1003]",nucleus_detection
17073,72270762727178308,"[14112, 123456, 3312]",349.543220,"[13216, 123328, 3093]","[15488, 125712, 3363]",nucleus_detection
17074,72200462366935570,"[8176, 127328, 2290]",390.043527,"[8080, 126608, 2245]","[10064, 129536, 2501]",nucleus_detection


## Update annotation table

In [ ]:
client.schema.schema_definition('nucleus_detection')

In [ ]:
# 20210902, volume column updated
# nuclei_aug2021
k=100
minidfs = [nuclei_df.loc[i:i+k-1, :] for i in range(0, len(nuclei_df), k)]
for dftmp in minidfs:
    client.annotation.update_annotation_df(table_name='nuclei_aug2021',
                                         df=dftmp,
                                         position_columns={'pt_position': 'pt', 'bb_start_position': 'bb_start', 'bb_end_position': 'bb_end'}, 
                                         aligned_volume_name='fanc_v4')

## Upload annotation table
make sure to create tables first

https://github.com/seung-lab/CAVEclient/blob/72ebf7180ca3c4195bec561e177c453554cc60c6/FlyWireAnnotationTutorial.ipynb

In [14]:
client.schema.schema_definition('nucleus_detection')

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'definitions': {'SpatialPoint': {'properties': {'position': {'type': 'array'}},
   'type': 'object',
   'required': ['position'],
   'additionalProperties': False},
  'BoundSpatialPoint': {'properties': {'position': {'type': 'array'},
    'root_id': {'type': 'integer'},
    'supervoxel_id': {'type': 'integer'}},
   'type': 'object',
   'required': ['position'],
   'additionalProperties': False},
  'NucleusDetection': {'properties': {'bb_end': {'type': 'object',
     '$ref': '#/definitions/SpatialPoint',
     'description': 'high corner of the bounding box'},
    'bb_start': {'type': 'object',
     '$ref': '#/definitions/SpatialPoint',
     'description': 'low corner of the bounding box'},
    'pt': {'type': 'object',
     '$ref': '#/definitions/BoundSpatialPoint',
     'description': 'the point to be used for attaching objects to the dynamic segmentation'},
    'valid': {'title': 'valid',
     'type': ['boolean', 'null'],
     'de

In [15]:
# get 502 error if you try to upload at the same time
# nuclei_aug2021
k=100
minidfs = [nuclei_df.loc[i:i+k-1, :] for i in range(0, len(nuclei_df), k)]
for dftmp in minidfs:
    client.annotation.post_annotation_df(table_name='nuclei_aug2021ver2',
                                         df=dftmp,
                                         position_columns={'pt_position': 'pt', 'bb_start_position': 'bb_start', 'bb_end_position': 'bb_end'}, 
                                         aligned_volume_name='fanc_v4')

In [ ]:
# soma_aug2021
k=100
minidfs = [soma_df.loc[i:i+k-1, :] for i in range(0, len(soma_df), k)]
for dftmp in minidfs:
#    client.annotation.post_annotation_df(table_name='soma_aug2021',
                                         df=dftmp,
                                         position_columns=['pt_position'], 
                                         aligned_volume_name='fanc_v4')

## Create tables

https://github.com/seung-lab/CAVEclient/blob/dec850dff9e0fa0a9ef8c2a0c79bf82cd2fc5b97/caveclient/annotationengine.py

In [12]:
auth.get_cv_path('nuclei_seg_Aug2021')['url']

'precomputed://gs://ng_scratch_ranl_zetta_30/vnc1_full_v3align_2/realigned_v1/seg/84214f67075692a9e41c02d164b466ba'

In [13]:
# client.annotation.create_table(table_name='nuclei_aug2021ver2',
                               schema_name='nucleus_detection',
                               description = 'an updated nucleus table with all the nuclei in FANC detected by CNN [created/uploaded by Sumiya Kuroda - sumiya-kuroda@g.ecc.u-tokyo.ac.jp]',
                               flat_segmentation_source=auth.get_cv_path('nuclei_seg_Aug2021')['url'],
                               voxel_resolution =[4.3, 4.3, 45.0])

<Response [200]>

In [ ]:
# client.annotation.create_table(table_name='soma_aug2021',
                               schema_name='nucleus_detection',
                               description = 'IDs shared with nuclei_aug2021. Based on nuclei segmentation at {}'.format(auth.get_cv_path('nuclei_seg_Aug2021')['url']),
                               voxel_resolution =[4.3, 4.3, 45.0])

## Test materialization

In [ ]:
client.materialize.homepage

In [ ]:
client.materialize.get_versions()

In [ ]:
# client.materialize.version = 21

In [ ]:
client.materialize.get_annotation_count('soma_aug2021')

In [ ]:
test=client.materialize.live_query('soma_aug2021',
                                   datetime.utcnow(),
                                   limit=20)

In [ ]:
test

In [ ]:
test=client.materialize.live_query('soma_aug2021',
                                   datetime.utcnow(),
                                   filter_in_dict={'id': [73115668492321345]},
                                   limit=10)

In [ ]:
test